In [0]:
df_product = spark.read.parquet("/mnt/bronze/production/product/20211208")
df_productcat = spark.read.parquet("/mnt/bronze/production/product_category/20211208")
df_productsubcat = spark.read.parquet("/mnt/bronze/production/product_subcategory/20211208")

In [0]:
from pyspark.sql.functions import *

In [0]:
display(df_product.limit(5))

ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
1,Adjustable Race,AR-5381,false,false,null,1000,750,0.0000,0.0000,null,null,null,null,0,null,null,null,null,null,2008-04-30T00:00:00.000+0000,null,null,694215b7-08f7-4c0d-acb1-d734ba44c0c8,2014-02-08T10:01:36.827+0000
2,Bearing Ball,BA-8327,false,false,null,1000,750,0.0000,0.0000,null,null,null,null,0,null,null,null,null,null,2008-04-30T00:00:00.000+0000,null,null,58ae3c20-4f3a-4749-a7d4-d568806cc537,2014-02-08T10:01:36.827+0000
3,BB Ball Bearing,BE-2349,true,false,null,800,600,0.0000,0.0000,null,null,null,null,1,null,null,null,null,null,2008-04-30T00:00:00.000+0000,null,null,9c21aed2-5bfa-4f18-bcb8-f11638dc2e4e,2014-02-08T10:01:36.827+0000
4,Headset Ball Bearings,BE-2908,false,false,null,800,600,0.0000,0.0000,null,null,null,null,0,null,null,null,null,null,2008-04-30T00:00:00.000+0000,null,null,ecfed6cb-51ff-49b5-b06c-7d8ac834db8b,2014-02-08T10:01:36.827+0000
316,Blade,BL-2036,true,false,null,800,600,0.0000,0.0000,null,null,null,null,1,null,null,null,null,null,2008-04-30T00:00:00.000+0000,null,null,e73e9750-603b-4131-89f5-3dd15ed5ff80,2014-02-08T10:01:36.827+0000


In [0]:
display(df_productcat.limit(5))

ProductCategoryID,Name,rowguid,ModifiedDate
1,Bikes,cfbda25c-df71-47a7-b81b-64ee161aa37c,2008-04-30T00:00:00.000+0000
2,Components,c657828d-d808-4aba-91a3-af2ce02300e9,2008-04-30T00:00:00.000+0000
3,Clothing,10a7c342-ca82-48d4-8a38-46a2eb089b74,2008-04-30T00:00:00.000+0000
4,Accessories,2be3be36-d9a2-4eee-b593-ed895d97c2a6,2008-04-30T00:00:00.000+0000


In [0]:
display(df_productsubcat.limit(5))

ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
1,1,Mountain Bikes,2d364ade-264a-433c-b092-4fcbf3804e01,2008-04-30T00:00:00.000+0000
2,1,Road Bikes,000310c0-bcc8-42c4-b0c3-45ae611af06b,2008-04-30T00:00:00.000+0000
3,1,Touring Bikes,02c5061d-ecdc-4274-b5f1-e91d76bc3f37,2008-04-30T00:00:00.000+0000
4,2,Handlebars,3ef2c725-7135-4c85-9ae6-ae9a3bdd9283,2008-04-30T00:00:00.000+0000
5,2,Bottom Brackets,a9e54089-8a1e-4cf5-8646-e3801f685934,2008-04-30T00:00:00.000+0000


In [0]:
df_product.registerTempTable("product")
df_productcat.registerTempTable("productcat")
df_productsubcat.registerTempTable("productsubcat")

In [0]:
df_join = spark.sql("""
select 
P.ProductID ProdutoID,
P.Name NomeProduto,
PSC.ProductCategoryID ProdutoCategoriaID,
PSC.Name NomeSubProduto,
PC.Name NomeCategoria
from product P
left join productsubcat PSC
  on P.ProductSubcategoryID = PSC.ProductSubcategoryID
left join productcat PC
  on PC.ProductCategoryID = PSC.ProductCategoryID""")

In [0]:
display(df_join.limit(5))

ProdutoID,NomeProduto,ProdutoCategoriaID,NomeSubProduto,NomeCategoria
1,Adjustable Race,null,null,null
2,Bearing Ball,null,null,null
3,BB Ball Bearing,null,null,null
4,Headset Ball Bearings,null,null,null
316,Blade,null,null,null


In [0]:
from pyspark.sql.functions import when
df2 = df_join.withColumn("NomeCategoria", when(df_join.NomeCategoria.isNull(), "Outros")
                                 .otherwise(df_join.NomeCategoria))
df2 = df2.withColumn("NomeSubProduto", when(df2.NomeSubProduto.isNull(), "Outros")
                                 .otherwise(df_join.NomeSubProduto))
df2 = df2.withColumn("ProdutoCategoriaID", when(df2.ProdutoCategoriaID.isNull(), -1)
                                 .otherwise(df2.ProdutoCategoriaID))

In [0]:
display(df2.limit(5))

ProdutoID,NomeProduto,ProdutoCategoriaID,NomeSubProduto,NomeCategoria
1,Adjustable Race,-1,Outros,Outros
2,Bearing Ball,-1,Outros,Outros
3,BB Ball Bearing,-1,Outros,Outros
4,Headset Ball Bearings,-1,Outros,Outros
316,Blade,-1,Outros,Outros


In [0]:
df2.registerTempTable("final")


In [0]:
sql = """
select
ProdutoID,
upper(NomeProduto) as NomeProduto,
ProdutoCategoriaID,
NomeSubProduto,
NomeCategoria
from
final"""
df_upper = spark.sql(sql)
display(df_upper.limit(5))

ProdutoID,NomeProduto,ProdutoCategoriaID,NomeSubProduto,NomeCategoria
1,ADJUSTABLE RACE,-1,Outros,Outros
2,BEARING BALL,-1,Outros,Outros
3,BB BALL BEARING,-1,Outros,Outros
4,HEADSET BALL BEARINGS,-1,Outros,Outros
316,BLADE,-1,Outros,Outros


In [0]:
df_upper.write.format("Delta").save("/mnt/silver/production/TBProdutoCategoria")